# 데이터 로드

In [322]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from labor_keyword")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

# 참조조문 전처리

In [323]:
df['참조조문']=df['참조조문'].fillna('참조조문 없음')

In [324]:
import re


def clean_clause(text): # 전부 , 단위로 분리될수 있게 변환

    text=text.replace('<br/>',',')

    # 각 그룹을 '/'로 나누어 처리
    groups = text.split('/')
    results = []

    for group in groups:
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        group=re.sub(r'(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\.', '', group) # 가., 나., 다., 라., 마., 바., 사., 아., 자., 차., 카., 타., 파., 하. 댜. 형태만 제거
        group=re.sub(r'(갸|냐|댜|랴|먀|뱌|샤|야|쟈|챠|캬|탸|퍄|햐)\.', '', group) # 가나다라의 오타도 제거
        
        group=re.sub(r'[\[\［]\d+[\]\］]\s*', '', group)  # '[1]', '[2]' 형식 제거
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        
        # 쉼표로 나눠 조각을 분리
        parts = re.split(r',\s*', group)                

        transformed_parts = []

        for part in parts:    # 조, 항, 호 뒤에 문자가 붙어 있는 경우 처리
            
            if re.search(r'([0-9]+조|[0-9]+항|[0-9]+호)([^\s])', part):
                part = re.sub(r'([0-9]+조|[0-9]+항|[0-9]+호)([^로의\s])', r'\1, \2', part)            
            part = part.strip()
            # 변환된 조문 추가
            transformed_parts.append(part)

        # 각 그룹을 쉼표로 연결하고 결과에 추가
        if transformed_parts:  # transformed_parts가 비어있지 않으면
            results.append(', '.join(transformed_parts))

    # 전체 결과를 반환
    return ', '.join(results)


In [325]:
ccl=df['참조조문'].apply(clean_clause)
ccl = ccl.apply(lambda x: x.replace(', ,', ',') if isinstance(x, str) else x)


In [326]:
def extract_law_names(series):
    law_names_set=set()
    pattern = r'(법|시행령|에\s*관한\s*법률|규칙|시행규칙|협정|의사록|관례|고시|대통령령|경제명령)' 
    # 정규 표현식을 사용하여 괄호를 기준으로 분리
    
    def split_outside_parentheses(text):
        result = []
        start = 0
        stack = []  # 괄호 스택

        for i, char in enumerate(text):
            if char in "{(":
                stack.append(char)  # 괄호 열기
            elif char in "})" and stack:
                stack.pop()  # 괄호 닫기
            elif char == "," and not stack:
                # 스택이 비어 있을 때의 쉼표는 분리 기준
                result.append(text[start:i].strip())
                start = i + 1

        # 마지막 남은 부분 추가
        result.append(text[start:].strip())
        return result

    for text in series:
        # 1. 쉼표로 문자열을 분리 (괄호 안 쉼표 무시)
        parts = split_outside_parentheses(text)
        print('파트분리:',parts)
        for part in parts:
            # 불필요한 \n 및 공백 제거
            part = re.sub(r'\n+', ' ', part)
            part = re.sub(r'\s+', ' ', part)  # 여러 공백을 하나로 줄임
            part = re.sub(r'[\[\]]', '', part)  # [] 괄호 제거
            part = re.sub(r'〔\d+〕', '', part)
            part = part.strip()
            part = re.sub(r'\{[^}]*\}', '', part)  # 중괄호 안의 내용 제거

            split_pattern = r'([^()]*)(\s*\([^()]*\))?'

            matches = re.findall(split_pattern, part)
            matches = [(off_maren.replace(',', '').strip(), maren.strip()) for off_maren, maren in matches]

            for match in matches:
                before_parentheses = match[0].strip()  # 괄호 이전 내용
                in_parentheses = match[1].strip() if match[1] else ''  # 괄호 내용
                # '숫자. ' 형식이 있다면 제거
                before_parentheses = re.sub(r'^\d+\.\s*', '', before_parentheses)
                print('before: ',before_parentheses)
                # print('in: ',in_parentheses)

                # 법 이름 추출: 중괄호가 제거된 `before_parentheses`에서 패턴 찾기
                match = re.search(pattern, before_parentheses)
                if match:
                    # 패턴이 일치하는 위치까지 추출
                    law_name = before_parentheses[:match.end()] + in_parentheses
                    if len(law_name) > 1:  # 길이가 1보다 큰 경우만 추가
                        law_names_set.add(law_name)

    return list(law_names_set)


# 결과 출력
law_names = extract_law_names(ccl)
print('추출된 법 이름:', law_names)

파트분리: ['근로기준법 제28조', '제29조', '노동위원회법 제23조', '근로기준법 제2조 제1항 제1호', '근로기준법 제2조 제1항 제1호', '제28조', '']
before:  근로기준법 제28조
before:  
before:  제29조
before:  
before:  노동위원회법 제23조
before:  
before:  근로기준법 제2조 제1항 제1호
before:  
before:  근로기준법 제2조 제1항 제1호
before:  
before:  제28조
before:  
before:  
파트분리: ['파견근로자 보호 등에 관한 법률 제6조의2 제1항', '민법 제390조', '제393조', '파견근로자 보호 등에 관한 법률 제6조의2 제1항', '민사소송법 제248조', '[소의 이익]', '민법 제162조 제1항', '']
before:  파견근로자 보호 등에 관한 법률 제6조의2 제1항
before:  
before:  민법 제390조
before:  
before:  제393조
before:  
before:  파견근로자 보호 등에 관한 법률 제6조의2 제1항
before:  
before:  민사소송법 제248조
before:  
before:  소의 이익
before:  
before:  민법 제162조 제1항
before:  
before:  
파트분리: ['구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호로 개정되기 전의 것) 제6조 제3항', '(현행 제6조의2 제1항 제3호, 제2항 참조)', '민법 제413조', '']
before:  구 파견근로자 보호 등에 관한 법률
before:  제6조 제3항
before:  
before:  
before:  
before:  민법 제413조
before:  
before:  
파트분리: ['파견근로자 보호 등에 관한 법률 제2조 제1호', '파견근로자 보호 등에 관한 법률 제2조 제1호', '파견근로자 보호 등에 관한 법률 제6조의2 제1항'

In [327]:
#최종 법이름
law_names=list(set(law_names))

In [328]:
law_names

['구 산업재해보상보험법(2008. 7. 1. 노동부령 제304호로 전문 개정되기 전의 것)',
 '구 산업재해보상보험법(2006. 8. 31. 노동부령 제258호로 개정되기 전의 것)',
 '구 고용보험법(2002. 12. 30. 법률 제6850호로 개정되기 전의 것)',
 '구 산업재해보상보험법',
 '구 지방재정법(2014. 5. 28. 법률 제12687호로 개정되기 전의 것)',
 '구 노인복지법(2019. 4. 1. 보건복지부령 제622호로 개정되기 전의 것)',
 '구 중소기업기본법(2007. 4. 11. 법률 제8360호로 전부 개정되기 전의 것)',
 '구 남녀고용평등법(2005. 5. 31. 법률 제7564호로 개정되기 전의 것)',
 '구 고용보험법(2003. 11. 29 대통령령 제18146호로 개정되기 전의 것)',
 '구 석탄산업법(1999. 2. 8. 법률 제5822호로 개정되기 전의 것)',
 '초ㆍ중등교육법',
 '부정경쟁방지법',
 '구 근로기준법(2007. 7. 27. 법률 제8561호로 개정되기 전의 것)',
 '구 모자보건법(1981. 11. 2. 대통령령 제10566호로 개정되기 전의 것)',
 '조세범처벌법',
 '구 자동차손해배상 보장법(2008. 3. 28. 법률 제9065호로 전부 개정되기 전의 것)',
 '구 한국해외개발공사법(1975.12.31. 법률 제2856호, )',
 '구 유아교육진흥법(1998. 9. 17. 법률 제5567호로 개정되기 전의 것)',
 '소득세법(라)',
 '구 고용보험법(2000. 12. 30. 대통령령 제17090호로 개정되기 전의 것)',
 '가정의례에 관한 법률(법률 제2604호, )',
 '구 공인노무사법(2020. 7. 28. 대통령령 제30873호로 개정되기 전의 것)',
 '구 산업재해보상보험법(2004. 10. 29. 대통령령 제18573호로 개정되기 전의 것)',
 '토지초과이득세법',
 '구 노동조합법(1996. 12. 31. 법률 제5244호로 폐지)',
 '진폐의 

In [329]:
# 법과 시행령 이름을 붙이는 함수
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = []  # 이전 법 이름 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정
                    break

            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_names):
                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x  # 문자열이 아닐 경우 그대로 반환

In [330]:
reason=ccl.apply(add_law_prefix)

In [331]:
reason

0       근로기준법 제28조, 근로기준법 제29조, 노동위원회법 제23조, 근로기준법 제2조...
1       파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민법 제390조, 민법 제393...
2       구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호로 개...
3       파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제...
4       근로기준법 제18조, 근로기준법 제18조, 구 고등교육법 시행령(2024. 2. 2...
                              ...                        
5007                                       소득세법(구) 제32조, 
5008                 근로기준법 제28조, 근로기준법 제94조, 근로기준법 제87조, 
5009         행정소송법(법률 제213호, 행정소송법 ) 제12조, 귀속재산처리법 제15조, 
5010                노동조합법 제33조, 노동조합법 제34조, 노동위원회규칙 제5조, 
5011                                           형법 제129조, 
Name: 참조조문, Length: 5012, dtype: object

# 참조판례 전처리

In [332]:
df['참조판례']=df['참조판례'].fillna('참조판례 없음')
#공백은 '참조판례 없음'으로 대체함.

#참조판례 정리
clean_part=[]
for part in df['참조판례']:
    part = part.strip()
    part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)  # 대괄호와 그 안의 내용 제거
    part = re.sub(r'\s*[가-힣]\.\s*', '', part)  # '가.', '나.', '다.'와 같은 패턴 제거
    part = part.strip('<br/>')  # 공백 제거
    part = part.replace('/', ',')  # '/'를 ','로 변경
    part = re.sub(r'\s+', ' ', part).strip()
    clean_part.append(part)

df['참조판례']=clean_part

# MYSQL에 넣기

In [333]:
dataF=pd.DataFrame(df['판례정보일련번호'])
dataF.head()
dataF['참조조문_1차전처리']=ccl
dataF['참조조문_전처리']=reason
dataF['참조판례_전처리']=df['참조판례']

dataF.to_csv('data/labor_case_clause_preprocessed.csv',index=False)

In [334]:
# db로 저장
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, TEXT
from db_info import user, password, host, port, database

####pandas dataframe에서는 불러올때 타입을 기본str로 불러와서 긴내용의 데이터삽입을 하려면 dtype을 설정해줘야함####
dtype = {
    '참조조문_1차전처리': TEXT,
    '참조조문_전처리': TEXT,
    '참조판례_전처리': TEXT
}
#판례내용_이유,판례내용_이유(전처리),판례내용_이유(불용어제거),클러스터,키워드

# DB정보
user = user
password = password
host = host
port = port
database = database

# CSV 파일 로드
df_case_clause_preprocessed = pd.read_csv('data/labor_case_clause_preprocessed.csv')

# MySQL 데이터베이스 연결 설정
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 데이터프레임을 MySQL 테이블에 저장
df_case_clause_preprocessed.to_sql('labor_case_clause_preprocessed', con=engine, if_exists='replace', index=False, dtype=dtype)
# 연결 닫기
engine.dispose()

print("데이터베이스 저장 완료")

데이터베이스 저장 완료
